IMPORTAMOS LIBRERÍAS

In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

IMPORTAMOS EL CSV DE UBEREATS QUE LIMPIAREMOS

In [6]:
ue = pd.read_csv("../Data/UberEats.csv", index_col = 0)
ue.sample(15)

,Restaurante,Valoración,Nota,Delivery_Fee,ETA,Tag,Precio,Dirección,Barrio
274,La Cuchara,40,3.5,2.90,40–50,Gourmet,€,"C/ Orense, 11, 28020",Tetuán
470,Bar y Kebab La Parada,23,3.0,1.40,15–25,Kebab,€,"Calle De Embajadores, 28, 28012",Carabanchel
396,Fast chicken chain,15,3.5,2.90,50–60,Pollo,€,"Calle De Sta Engracia, 108, 28003",Moncloa-Aravaca
366,Super naan,15,3.5,1.40,60–70,India,€,"Calle Gran Vía, 16, 28004",Chamberí
581,Asador Delicasa,14,4.0,1.40,25–35,Carne,€,"Calle De Gutenberg 3, 28014",Puente de Vallecas
526,Alice Pizza,23,3.6,2.50,40–60,Pizza,€,"Calle De Méndez Álvaro 77, 28045",Usera
53,Cerveceria Riaño,28,3.1,2.90,45–55,Cerveza,€,"Calle De José Calvo 6, 28039",Centro
576,Comida Fusión,19,3.7,1.40,30–40,Fusión latina,€,"Avenida De Peña Prieta 22, 28038",Puente de Vallecas
535,Kaleka Poke,23,3.6,0.90,25–35,Kebab,€,"Paseo Molino, 7, 28045",Usera
296,Pollo Criollo,40,3.5,2.40,55–65,Pollo,€,"Calle De Lérida 58, 28020",Tetuán


In [32]:
for col in ue:
    print(col)
    print(" ")
    print(ue[col].value_counts())
    print("-" * 30)

Restaurante
 
Empanadas BAIRES    13
Papa John's         13
McDonald's          12
Popeyes             11
Arepish             11
                    ..
Popizza              1
La Manousheria       1
Burger Las Vegas     1
Crepes & Waffles     1
Filippo Pizza        1
Name: Restaurante, Length: 251, dtype: int64
------------------------------
Valoración
 
200    84
23     48
30     38
15     37
40     37
       ..
194     1
78      1
145     1
63      1
80      1
Name: Valoración, Length: 82, dtype: int64
------------------------------
Nota
 
4.3    60
4.2    60
4.6    56
3.5    53
4.4    51
3.1    48
4.5    44
2.9    43
4.1    42
3.6    42
3.8    36
3.0    22
4.8    21
2.7    21
4.0    17
3.9    16
3.7    13
4.9    12
2.8    11
4.7    11
5.0     4
3.2     4
3.4     2
3.3     1
Name: Nota, dtype: int64
------------------------------
Delivery_Fee
 
2.40    165
1.90    131
1.40    122
0.90     85
2.90     67
2.50     27
0.40     19
1.00     12
3.00     12
4.25     11
3.40     11
2.99      

In [4]:
barrio = pd.read_csv("../Data/Barrios_cifras.csv")
barrio.head(13)

,Superficie,Superficie.1,Densidad,Población,0 a 15,16 a 64,65 +,% 0 a 15,% 16 a 64,% 65 +,Esp,Extr,% Esp,% Extr,Nacim,Def,C Natural,€ m2
0,Centro,523.000,270,141.236,10.608,108.667,21.961,8%,77%,16%,104.726,36.509,74%,26%,863.000,1.252,-389.000,4.865
1,Arganzuela,646.000,239,154.243,18.571,105.078,30.594,12%,68%,20%,137.604,16.637,89%,11%,1.142,1.764,-622.000,3.888
2,Retiro,546.000,217,118.557,14.443,73.037,31.077,12%,62%,26%,108.277,10.279,91%,9%,801.000,1.627,-826.000,4.502
3,Salamanca,539.000,271,146.016,16.147,95.122,34.747,11%,65%,24%,124.068,21.948,85%,15%,1.150,1.937,-787.000,5.740
4,Chamartín,917.000,161,145.700,19.913,91.663,34.124,14%,63%,23%,130.641,15.059,90%,10%,1.305,1.899,-594.000,4.999
5,Tetuán,537.000,297,159.849,18.293,111.262,30.294,11%,70%,19%,127.202,32.647,80%,20%,1.353,1.984,-631.000,3.649
6,Chamberí,467.000,296,138.667,14.767,90.307,33.593,11%,65%,24%,121.264,17.403,87%,13%,1.076,1.785,-709.000,5.283
7,Moncloa-Aravaca,4.654,26,120.834,17.036,77.360,26.438,14%,64%,22%,106.834,14.000,88%,12%,936.000,1.485,-549.000,3.912
8,Carabanchel,1.405,184,258.633,36.742,173.880,48.011,14%,67%,19%,202.322,56.311,78%,22%,1.906,3.057,-1.151,2.165
9,Usera,778.000,183,142.454,21.188,97.802,23.464,15%,69%,16%,107.442,35.010,75%,25%,1.124,1.554,-430.000,1.942
